# Brain MRI Detection using different Machine Learning Models

## Team Members
- Mohammad Ali Zahir
- Marwa Khalid

## Required Librairies
**Run the cell below to install all the required libraries.**